In [1]:
# Familiar imports
import numpy as np
import pandas as pd
import random
import os
import time
from pathlib import Path

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

#import lightgbm as lgb
#import xgboost as xgb
import catboost as ctb

import warnings
warnings.simplefilter('ignore')

# Parameters

In [2]:
target = 'target'

DEBUG = False

if DEBUG:
    N_ESTIMATORS = 1
    N_SPLITS = 2
    SEED = 17
    CVSEED = 17
    EARLY_STOPPING_ROUNDS = 1
    VERBOSE = 100
    #N_ITERS = 2
else:
    N_SPLITS = 10
    N_ESTIMATORS = 20000
    EARLY_STOPPING_ROUNDS = 200
    VERBOSE = 1000
    SEED = 17
    CVSEED = 17
    #N_ITERS = 10

In [3]:
def set_seed(seed=17):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
set_seed(SEED)

# Load data

In [4]:
# Load the training data
train = pd.read_csv("../input/30-days-of-ml/train.csv")
test = pd.read_csv("../input/30-days-of-ml/test.csv")

# Preview the data
train.head()

,id,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,target
0,1,B,B,B,C,B,B,A,E,C,...,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850,8.113634
1,2,B,B,A,A,B,D,A,F,A,...,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083,8.481233
2,3,A,A,A,C,B,D,A,D,A,...,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846,8.364351
3,4,B,B,A,C,B,D,A,E,C,...,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682,8.049253
4,6,A,A,A,C,B,D,A,E,A,...,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823,7.972260


In [5]:
# Separate target from features
y = train['target']
features = train.drop(['id','target'], axis=1)

# Preview features
features.head()

,cat0,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13
0,B,B,B,C,B,B,A,E,C,N,...,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,0.237281,0.377873,0.322401,0.869850
1,B,B,A,A,B,D,A,F,A,O,...,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,0.906013,0.921701,0.261975,0.465083
2,A,A,A,C,B,D,A,D,A,F,...,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,0.748809,0.620126,0.541474,0.763846
3,B,B,A,C,B,D,A,E,C,K,...,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,0.346010,0.714610,0.540150,0.280682
4,A,A,A,C,B,D,A,E,A,N,...,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,1.000773,0.776742,0.625849,0.250823


# Preprocessing

In [6]:
'''
# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# ordinal-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()
ordinal_encoder = OrdinalEncoder()
X[object_cols] = ordinal_encoder.fit_transform(features[object_cols])
X_test[object_cols] = ordinal_encoder.transform(test[object_cols])

# Preview the ordinal-encoded features
X.head()
'''

"\n# List of categorical columns\nobject_cols = [col for col in features.columns if 'cat' in col]\n\n# ordinal-encode categorical columns\nX = features.copy()\nX_test = test.drop(['id'], axis=1).copy()\nordinal_encoder = OrdinalEncoder()\nX[object_cols] = ordinal_encoder.fit_transform(features[object_cols])\nX_test[object_cols] = ordinal_encoder.transform(test[object_cols])\n\n# Preview the ordinal-encoded features\nX.head()\n"

In [7]:

# List of categorical columns
object_cols = [col for col in features.columns if 'cat' in col]

# one-hot-encode categorical columns
X = features.copy()
X_test = test.drop(['id'], axis=1).copy()

oh_encoder = OneHotEncoder(sparse=False, handle_unknown="ignore")
X_ohe = oh_encoder.fit_transform(features[object_cols])
X_test_ohe = oh_encoder.transform(test[object_cols])

X_ohe = pd.DataFrame(X_ohe, columns=[f"ohe_{i}" for i in range(X_ohe.shape[1])])
X_test_ohe = pd.DataFrame(X_test_ohe, columns=[f"ohe_{i}" for i in range(X_test_ohe.shape[1])])

X = pd.concat([X, X_ohe], axis=1)
X_test = pd.concat([X_test, X_test_ohe], axis=1)
X = X.drop(object_cols, axis=1)
X_test = X_test.drop(object_cols, axis=1)
    
# Preview the one-hot-encoded features
X.head()


,cont0,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,...,ohe_46,ohe_47,ohe_48,ohe_49,ohe_50,ohe_51,ohe_52,ohe_53,ohe_54,ohe_55
0,0.201470,-0.014822,0.669699,0.136278,0.610706,0.400361,0.160266,0.310921,0.389470,0.267559,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.743068,0.367411,1.021605,0.365798,0.276853,0.533087,0.558922,0.516294,0.594928,0.341439,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,0.742708,0.310383,-0.012673,0.576957,0.285074,0.650609,0.375348,0.902567,0.555205,0.843531,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.429551,0.620998,0.577942,0.280610,0.284667,0.668980,0.239061,0.732948,0.679618,0.574844,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,1.058291,0.367492,-0.052389,0.232407,0.287595,0.686964,0.420667,0.648182,0.684501,0.956692,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [8]:
scale_features = [col for col in features.columns if 'cont' in col]

ss = StandardScaler()
X[scale_features] = ss.fit_transform(features[scale_features])
X_test[scale_features] = ss.transform(test[scale_features])

In [9]:
# Swap noise

# Random
def apply_noise_rn(df, p=.75):
    should_not_swap = np.random.binomial(1, p, df.shape)
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Row-wise
def apply_noise_row(df, p=.75):
    should_not_swap = np.zeros(df.shape)
    for i in range(df.shape[0]):
        for j in np.random.choice(df.shape[1],int(p*df.shape[1]),replace=False):
            should_not_swap[i,j]=1 
    corrupted_df = df.where(should_not_swap == 1, np.random.permutation(df))
    return corrupted_df

# Optuna

In [10]:
# Optuna for parameter search
!pip install -q optuna

import optuna
import pickle

In [11]:
# for the fixed learning rate, use the opt n iterations and tune the tree hyperparameters
def objective(trial, X=X, y=y):
  """
  """
  param_space = {
               'bootstrap_type': trial.suggest_categorical('bootstrap_type', ['Bernoulli', 'MVS']),
            'loss_function': 'RMSE',
            'eval_metric': 'RMSE',
          #'task_type': 'GPU',
          'max_depth':trial.suggest_int('max_depth', 3, 5),
           'learning_rate':trial.suggest_uniform('learning_rate', 9.5e-3, 1e-2),
          'n_estimators':N_ESTIMATORS,
        'max_bin':trial.suggest_int('max_bin', 464, 484),
             'min_data_in_leaf':trial.suggest_int('min_data_in_leaf', 77, 83),
         'reg_lambda':trial.suggest_uniform('reg_lambda', 0.735, 0.785),
            'subsample': trial.suggest_uniform('subsample', 0.896, 0.917)
                }
            
  seed_list=[SEED, SEED+1]
  #kf = KFold(n_splits=N_SPLITS, shuffle=True, random_state=CVSEED)
  #lgb_oof = np.zeros(train.shape[0])
  #lgb_pred = np.zeros(test.shape[0])
  X_tr, X_va, y_tr, y_va = train_test_split(X,y,test_size=.2,random_state=CVSEED)
  pred=np.zeros(y_va.shape[0])
    
  #for (trn_idx, val_idx) in enumerate(kf.split(X, y)):
  #print(f"===== fold {fold} =====")
    
  #X_tr = apply_noise_row(X_tr)

  
  #start = time.time()
  for inseed in seed_list:
    param_space['random_seed'] = inseed

    model =  ctb.CatBoostRegressor(**param_space)
    model.fit(
        X_tr, 
        y_tr,
        eval_set=[(X_va, y_va)],
         use_best_model=True,
        early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        verbose=VERBOSE,
        )


    pred += model.predict(X_va) / len(seed_list)
    
  #elapsed = time.time() - start
  rmse = mean_squared_error(y_va, pred, squared=False)
  #print(f"fold {fold} - lgb rmse: {rmse:.6f}, elapsed time: {elapsed:.2f}sec\n")            
  
  #print(f"oof lgb_rmse = {mean_squared_error(train[target], lgb_oof, squared=False)}")

  
  return rmse

In [12]:
study = optuna.create_study(direction='minimize')
study.optimize(objective,n_trials= 30)
print('Number of finished trials:', len(study.trials))
print('Best trial:', study.best_trial.params)

[I 2021-08-27 07:18:39,680] A new study created in memory with name: no-name-ed1f4520-f0ac-4f40-9a74-2570d6aad262


0:	learn: 0.7471868	test: 0.7435984	best: 0.7435984 (0)	total: 77.9ms	remaining: 25m 57s
1000:	learn: 0.7331977	test: 0.7298734	best: 0.7298734 (1000)	total: 20.1s	remaining: 6m 21s
2000:	learn: 0.7291776	test: 0.7264020	best: 0.7264020 (2000)	total: 38.8s	remaining: 5m 49s
3000:	learn: 0.7266134	test: 0.7243249	best: 0.7243249 (3000)	total: 58.1s	remaining: 5m 29s
4000:	learn: 0.7245456	test: 0.7225821	best: 0.7225821 (4000)	total: 1m 17s	remaining: 5m 10s
5000:	learn: 0.7228409	test: 0.7212598	best: 0.7212598 (4999)	total: 1m 36s	remaining: 4m 50s
6000:	learn: 0.7213924	test: 0.7201800	best: 0.7201800 (6000)	total: 1m 56s	remaining: 4m 31s
7000:	learn: 0.7201939	test: 0.7193712	best: 0.7193712 (7000)	total: 2m 15s	remaining: 4m 11s
8000:	learn: 0.7191315	test: 0.7186844	best: 0.7186838 (7999)	total: 2m 35s	remaining: 3m 52s
9000:	learn: 0.7181798	test: 0.7181215	best: 0.7181212 (8999)	total: 2m 55s	remaining: 3m 33s
10000:	learn: 0.7173296	test: 0.7176874	best: 0.7176874 (10000)	tota

[I 2021-08-27 07:31:42,988] Trial 0 finished with value: 0.7156520349331383 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 3, 'learning_rate': 0.009650611348117255, 'max_bin': 464, 'min_data_in_leaf': 78, 'reg_lambda': 0.7386244243668113, 'subsample': 0.9137679370445592}. Best is trial 0 with value: 0.7156520349331383.


0:	learn: 0.7471874	test: 0.7435990	best: 0.7435990 (0)	total: 33.3ms	remaining: 11m 5s
1000:	learn: 0.7332493	test: 0.7299147	best: 0.7299147 (1000)	total: 20.1s	remaining: 6m 21s
2000:	learn: 0.7292256	test: 0.7264159	best: 0.7264159 (2000)	total: 39.1s	remaining: 5m 51s
3000:	learn: 0.7266214	test: 0.7242760	best: 0.7242760 (3000)	total: 58.6s	remaining: 5m 31s
4000:	learn: 0.7246117	test: 0.7226122	best: 0.7226122 (4000)	total: 1m 18s	remaining: 5m 12s
5000:	learn: 0.7228984	test: 0.7212609	best: 0.7212608 (4999)	total: 1m 37s	remaining: 4m 52s
6000:	learn: 0.7214197	test: 0.7201596	best: 0.7201596 (6000)	total: 1m 57s	remaining: 4m 33s
7000:	learn: 0.7202141	test: 0.7193413	best: 0.7193413 (7000)	total: 2m 16s	remaining: 4m 12s
8000:	learn: 0.7191775	test: 0.7186933	best: 0.7186933 (8000)	total: 2m 35s	remaining: 3m 53s
9000:	learn: 0.7182298	test: 0.7181242	best: 0.7181242 (8999)	total: 2m 55s	remaining: 3m 34s
10000:	learn: 0.7173936	test: 0.7176934	best: 0.7176934 (10000)	total

[I 2021-08-27 07:44:48,176] Trial 1 finished with value: 0.7156357166479694 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 3, 'learning_rate': 0.009556755012781432, 'max_bin': 483, 'min_data_in_leaf': 77, 'reg_lambda': 0.7581367961791012, 'subsample': 0.9063122321329882}. Best is trial 1 with value: 0.7156357166479694.


...

0:	learn: 0.7471731	test: 0.7435808	best: 0.7435808 (0)	total: 29.9ms	remaining: 9m 57s
1000:	learn: 0.7314193	test: 0.7283137	best: 0.7283137 (1000)	total: 21.2s	remaining: 6m 41s
2000:	learn: 0.7269589	test: 0.7246405	best: 0.7246405 (2000)	total: 41.5s	remaining: 6m 13s
3000:	learn: 0.7239648	test: 0.7223618	best: 0.7223618 (3000)	total: 1m 1s	remaining: 5m 50s
4000:	learn: 0.7216754	test: 0.7207570	best: 0.7207570 (4000)	total: 1m 22s	remaining: 5m 30s
5000:	learn: 0.7198309	test: 0.7196245	best: 0.7196243 (4999)	total: 1m 43s	remaining: 5m 10s
6000:	learn: 0.7182172	test: 0.7187419	best: 0.7187419 (6000)	total: 2m 3s	remaining: 4m 49s
7000:	learn: 0.7167786	test: 0.7180550	best: 0.7180550 (7000)	total: 2m 24s	remaining: 4m 28s
8000:	learn: 0.7154734	test: 0.7175833	best: 0.7175833 (8000)	total: 2m 45s	remaining: 4m 8s
9000:	learn: 0.7142419	test: 0.7171601	best: 0.7171601 (9000)	total: 3m 5s	remaining: 3m 46s
10000:	learn: 0.7131040	test: 0.7168368	best: 0.7168368 (10000)	total: 3

[I 2021-08-27 13:47:37,882] Trial 28 finished with value: 0.7153641724037562 and parameters: {'bootstrap_type': 'Bernoulli', 'max_depth': 4, 'learning_rate': 0.009841091297455399, 'max_bin': 480, 'min_data_in_leaf': 81, 'reg_lambda': 0.7811528984119417, 'subsample': 0.9052515558785172}. Best is trial 21 with value: 0.715286836879976.


0:	learn: 0.7471869	test: 0.7435985	best: 0.7435985 (0)	total: 24.9ms	remaining: 8m 17s
1000:	learn: 0.7332283	test: 0.7298801	best: 0.7298801 (1000)	total: 20.4s	remaining: 6m 26s
2000:	learn: 0.7292138	test: 0.7264170	best: 0.7264170 (2000)	total: 40.4s	remaining: 6m 3s
3000:	learn: 0.7265916	test: 0.7242454	best: 0.7242454 (3000)	total: 59.8s	remaining: 5m 38s
4000:	learn: 0.7245431	test: 0.7225247	best: 0.7225247 (4000)	total: 1m 19s	remaining: 5m 18s
5000:	learn: 0.7228694	test: 0.7212265	best: 0.7212264 (4998)	total: 1m 39s	remaining: 4m 58s
6000:	learn: 0.7213988	test: 0.7201496	best: 0.7201496 (6000)	total: 1m 59s	remaining: 4m 38s
7000:	learn: 0.7201938	test: 0.7193250	best: 0.7193250 (7000)	total: 2m 18s	remaining: 4m 17s
8000:	learn: 0.7191409	test: 0.7186519	best: 0.7186519 (8000)	total: 2m 38s	remaining: 3m 57s
9000:	learn: 0.7181955	test: 0.7180935	best: 0.7180934 (8999)	total: 2m 58s	remaining: 3m 37s
10000:	learn: 0.7173467	test: 0.7176830	best: 0.7176830 (10000)	total:

[I 2021-08-27 14:00:50,771] Trial 29 finished with value: 0.7156341770217636 and parameters: {'bootstrap_type': 'MVS', 'max_depth': 3, 'learning_rate': 0.009614368251215283, 'max_bin': 484, 'min_data_in_leaf': 79, 'reg_lambda': 0.7702542369641678, 'subsample': 0.8984901997087282}. Best is trial 21 with value: 0.715286836879976.


Number of finished trials: 30
Best trial: {'bootstrap_type': 'Bernoulli', 'max_depth': 4, 'learning_rate': 0.009794447435602701, 'max_bin': 481, 'min_data_in_leaf': 83, 'reg_lambda': 0.7810869504935302, 'subsample': 0.9037185505230915}


In [13]:
study.best_params

{'bootstrap_type': 'Bernoulli',
 'max_depth': 4,
 'learning_rate': 0.009794447435602701,
 'max_bin': 481,
 'min_data_in_leaf': 83,
 'reg_lambda': 0.7810869504935302,
 'subsample': 0.9037185505230915}

# Log

====== Ordinal encoding =========

0.7173727937026717 no noise ver2

====== One-hot encoding =========

0.7158375044779342 no noise ver1

0.7155633135916154 no noise ver3 (narrow space)

0.7153157917508417 no noise ver4 (narrow space)

0.7152896009256621 no noise ver5 (narrow space)